### Imports

In [31]:
import pandas as pd
import kagglehub
from sklearn.preprocessing import StandardScaler

# MERGING

In [32]:
# Download latest version
path = kagglehub.dataset_download("sujaykapadnis/nfl-stadium-attendance-dataset")

# Load the dataframe from the file
def load_data(name):
    return pd.read_csv(f"{path}/{name}")

attendance_df = load_data("attendance.csv")
standings_df = load_data("standings.csv")
games_df = load_data("games.csv")

attendance_weekly_df = attendance_df[['team', 'team_name', 'year', 'week', 'weekly_attendance']]

attendance_df = attendance_df.drop(columns=['weekly_attendance'])

### ATTENDANCE
# year was shitted by one year, so that last years standings will have influence on attendance
attendance_df['year'] = attendance_df['year'] + 1

attendance_df = attendance_df.merge(attendance_weekly_df, on=['team', 'team_name', 'year', 'week'], how='left')


### STANDINGS
# similar like above for standings
standings_df['year'] = standings_df['year'] + 1

attendance_standings_df = pd.merge(attendance_df, standings_df, on=['team', 'team_name', 'year'])

attendance_standings_df['team_name'] = attendance_standings_df['team'] + ' ' + attendance_standings_df['team_name']

### GAMES
games_df = games_df[games_df['week'].str.isnumeric()]
games_df['week'] = games_df['week'].astype(int)

### MERGE

df1 = attendance_standings_df.merge(games_df, left_on=['year', 'week', 'team_name'], right_on=['year', 'week', 'home_team'], how='inner')
df2 = attendance_standings_df.merge(games_df, left_on=['year', 'week', 'team_name'], right_on=['year', 'week', 'away_team'], how='inner')

df = df1.merge(df2, on=['year', 'week', 'home_team', 'away_team'], how='inner', suffixes=('_home', '_away'))

df = df.drop(columns=['home_team', 'away_team', 'weekly_attendance_away', 'team_away'])
df = df.rename(columns={'team_home': 'city'})

# PREPROCESS

In [33]:
from sklearn.preprocessing import StandardScaler

duplicated_columns = ['winner', 'tie', 'day', 'date', 'time', 'pts_win', 'pts_loss', 'yds_win', 'turnovers_win', 'yds_loss', 'turnovers_loss', 'home_team_name', 'home_team_city', 'away_team_name', 'away_team_city', 'loss']
duplicated_columns_away = list(map(lambda x: x + '_away', duplicated_columns))
duplicated_columns_home = list(map(lambda x: x + '_home', duplicated_columns))

duplicated_columns_mapping = dict(zip(duplicated_columns_home, duplicated_columns))

df = df.drop(columns=duplicated_columns_away)
df = df.rename(columns=duplicated_columns_mapping)

columns_to_drop = ['total_home', 'away_home', 'date', 'time', 'city']

columns_to_rename = {
    'home_home': 'home_attendance_last_year_home',
    'away_home': 'away_attendance_last_year_home',
    'home_away': 'home_attendance_last_year_away',
    'away_away': 'away_attendance_last_year_away',
}

df = df.drop(columns=columns_to_drop)
df = df.rename(columns=columns_to_rename)
df = df.replace({ 'Playoffs': 1, 'No Playoffs': 0, 'Won Superbowl': 1, 'No Superbowl': 0 })

df[['playoffs_away', 'sb_winner_away', 'playoffs_home', 'sb_winner_home']] = df[['playoffs_away', 'sb_winner_away', 'playoffs_home', 'sb_winner_home']].astype(int)


season_stats_df = pd.DataFrame(columns=['year', 'week', 'team_name', 'points', 'yards', 'turnovers', 'win', 'loss', 'tie'], dtype=int)
season_stats_df['team_name'] = season_stats_df['team_name'].astype(str)

for index, row in df.iterrows():
    winning_team = row['winner']    
    losing_team = row['team_name_away'] if winning_team == row['team_name_home'] else row['team_name_home']

    for team in [winning_team, losing_team]:
        season_stats_df.loc[len(season_stats_df)] = {
            'year': row['year'],
            'week': row['week'],
            'team_name': team,
            'points': row['pts_win'] if team == winning_team else row['pts_loss'],
            'yards': row['yds_win'] if team == winning_team else row['yds_loss'],
            'turnovers': row['turnovers_win'] if team == winning_team else row['turnovers_loss'],
            'win': 1 if team == winning_team and pd.isna(row['tie']) else 0,
            'loss': 1 if team == losing_team and pd.isna(row['tie']) else 0,
            'tie': 1 if not pd.isna(row['tie'])  else 0
        }

min_year = season_stats_df['year'].min()
max_year = season_stats_df['year'].max()
teams = season_stats_df['team_name'].unique()

# Fill in missing weeks with 0s
for team in teams:
    for year in range(min_year, max_year + 1):
        for week in range(1, 18):
            if len(season_stats_df[(season_stats_df['team_name'] == team) & (season_stats_df['year'] == year) & (season_stats_df['week'] == week)]) == 0:
                season_stats_df.loc[len(season_stats_df)] = {
                    'year': year,
                    'week': week,
                    'team_name': team,
                    'points': 0,
                    'yards': 0,
                    'turnovers': 0,
                    'win': 0,
                    'loss': 0,
                    'tie': 0
                }


# Group by years, and calculate running totals
season_stats_df = season_stats_df.sort_values(by=['team_name', 'year', 'week'])
season_stats_df['points'] = season_stats_df['points'].astype(int)
season_stats_df['yards'] = season_stats_df['yards'].astype(int)
season_stats_df['turnovers'] = season_stats_df['turnovers'].astype(int)
season_stats_df['win'] = season_stats_df['win'].astype(int)
season_stats_df['loss'] = season_stats_df['loss'].astype(int)
season_stats_df['tie'] = season_stats_df['tie'].astype(int)

season_stats_df['points'] = season_stats_df.groupby(['team_name', 'year'])['points'].cumsum()
season_stats_df['yards'] = season_stats_df.groupby(['team_name', 'year'])['yards'].cumsum()
season_stats_df['turnovers'] = season_stats_df.groupby(['team_name', 'year'])['turnovers'].cumsum()
season_stats_df['win'] = season_stats_df.groupby(['team_name', 'year'])['win'].cumsum()
season_stats_df['loss'] = season_stats_df.groupby(['team_name', 'year'])['loss'].cumsum()
season_stats_df['tie'] = season_stats_df.groupby(['team_name', 'year'])['tie'].cumsum()


season_stats_df['week'] = season_stats_df['week'] + 1
# season_stats_df = season_stats_df[season_stats_df['week'] <= 17]

for index, row in season_stats_df[season_stats_df['week'] == 18].iterrows():
    season_stats_df.loc[index] = {
        'year': row['year'],
        'week': 1,
        'team_name': row['team_name'],
        'points': 0,
        'yards': 0,
        'turnovers': 0,
        'win': 0,
        'loss': 0,
        'tie': 0
    }

season_stats_df = season_stats_df.sort_values(by=['team_name', 'year', 'week'])
season_stats_df[(season_stats_df['team_name'] == 'Arizona Cardinals')][season_stats_df['year'] == 2019]
games_columns_to_delete = ['winner', 'tie', 'day', 'pts_win', 'pts_loss', 'yds_win', 'turnovers_win', 'yds_loss', 'turnovers_loss', 'home_team_name', 'home_team_city', 'away_team_name', 'away_team_city']
df = df.drop(columns=games_columns_to_delete)

season_stats_df = season_stats_df.rename(columns={ 'team_name': 'team_name_home' })
df = df.merge(season_stats_df, on=['team_name_home', 'year', 'week'], how='inner', suffixes=['', '_home'])

season_stats_df = season_stats_df.rename(columns={ 'team_name_home': 'team_name_away' })
df = df.merge(season_stats_df, on=['team_name_away', 'year', 'week'], how='inner', suffixes=['', '_away'])

/var/folders/db/djqfj81d1vs_17ntzp2n867m0000gn/T/ipykernel_41993/2035849910.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({ 'Playoffs': 1, 'No Playoffs': 0, 'Won Superbowl': 1, 'No Superbowl': 0 })
/var/folders/db/djqfj81d1vs_17ntzp2n867m0000gn/T/ipykernel_41993/2035849910.py:104: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  season_stats_df[(season_stats_df['team_name'] == 'Arizona Cardinals')][season_stats_df['year'] == 2019]


## Creating Attendance IDs for Splitting the Data into Train, Validation and Test for DBRepo

In [34]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)


df['attendance_id'] = df.index


cols = ['attendance_id'] + [col for col in df.columns if col != 'attendance_id']
df = df[cols]


In [35]:
df.to_csv('input.csv', index=False)


In [36]:
df

,attendance_id,team_name_home,year,home_attendance_last_year_home,week,weekly_attendance_home,wins_home,loss,points_for_home,points_against_home,...,turnovers,win,loss_home,tie,points_away,yards_away,turnovers_away,win_away,loss_away,tie_away
0,0,Pittsburgh Steelers,2006,507434,1,64927.0,11,5,389,258,...,0,0,0,0,0,0,0,0,0,0
1,1,Minnesota Vikings,2012,502529,8,60860.0,3,13,340,449,...,11,5,2,0,148,2080,7,2,4,0
2,2,Cincinnati Bengals,2004,479488,7,65806.0,8,8,346,384,...,11,1,4,0,130,2224,8,5,1,0
3,3,Buffalo Bills,2005,574399,12,71440.0,9,7,395,284,...,14,4,6,0,253,3026,19,7,3,0
4,4,Atlanta Falcons,2001,422814,2,47804.0,4,12,252,413,...,0,0,1,0,24,291,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4803,4803,Los Angeles Chargers,2018,202687,5,25362.0,9,7,355,272,...,5,2,2,0,97,1767,7,1,3,0
4804,4804,Tampa Bay Buccaneers,2002,524468,1,65554.0,9,7,324,280,...,0,0,0,0,0,0,0,0,0,0
4805,4805,Chicago Bears,2013,498633,10,62431.0,10,6,375,277,...,11,5,3,0,217,3328,12,5,3,0
4806,4806,San Francisco 49ers,2015,566192,9,70799.0,8,8,306,340,...,7,2,6,0,213,3318,14,6,2,0
